In [2]:
# Retrain 3 Models on Full Train (202 samples) with Pretrained Weights & Lighter Reg (Expert Pivot)
# Goal: Better generalization via transfer learning (target holdout WLL <=0.42, LB <=0.45 medal)
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import GradScaler, autocast
import timm
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from sklearn.metrics import log_loss
from scipy.special import expit as sigmoid
import os
from tqdm import tqdm
import time
import gc
import warnings
warnings.filterwarnings('ignore')

# Seeding
def seed_everything(seed):
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Clear GPU cache before training
gc.collect()
torch.cuda.empty_cache()
print(f'GPU memory after clear: {torch.cuda.memory_allocated()/1e9:.1f}GB allocated, {torch.cuda.memory_reserved()/1e9:.1f}GB reserved')

# Load full train_mips.csv (202 samples)
train_df = pd.read_csv('data/train_mips.csv')
print(f'Full train size: {len(train_df)}')
label_cols = ['C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7']
y_full = train_df[label_cols].values

# Dataset
class MIPDataset(Dataset):
    def __init__(self, df, mip_dir, labels=None, transform=None):
        self.df = df.reset_index(drop=True)
        self.mip_dir = mip_dir
        self.labels = labels
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        uid = self.df.iloc[idx]['StudyInstanceUID']
        mip = np.load(os.path.join(self.mip_dir, f'{uid}.npy')).astype(np.float32)
        img = np.transpose(mip, (1, 2, 0))
        if self.transform:
            img = self.transform(image=img)['image']
        else:
            img = (img - 0.5) / 0.5
            img = torch.from_numpy(img).permute(2, 0, 1).float()
        if self.labels is not None:
            label = torch.tensor(self.labels[idx], dtype=torch.float32)
            return img, label
        return img

# Lighter, anatomy-safe transforms (320x320 to save memory)
IMG_SIZE = 320
train_transform = A.Compose([
    A.RandomResizedCrop(size=(IMG_SIZE, IMG_SIZE), scale=(0.9, 1.0), p=1.0),
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=10, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.Normalize(mean=0.5, std=0.5),
    ToTensorV2()
])
val_transform = A.Compose([A.Resize(IMG_SIZE, IMG_SIZE), A.Normalize(mean=0.5, std=0.5), ToTensorV2()])

# Model builders with pretrained & moderate dropout
def build_convnext_v1(seed=42):
    seed_everything(seed)
    return timm.create_model('convnext_tiny', pretrained=True, num_classes=7, in_chans=3,
                             drop_rate=0.25, drop_path_rate=0.1).to(device)

def build_convnext_v2(seed=123):
    seed_everything(seed)
    return timm.create_model('convnext_tiny', pretrained=True, num_classes=7, in_chans=3,
                             drop_rate=0.25, drop_path_rate=0.1).to(device)

def build_regnet(seed=456):
    seed_everything(seed)
    return timm.create_model('regnety_004', pretrained=True, num_classes=7, in_chans=3,
                             drop_rate=0.25, drop_path_rate=0.1).to(device)

# 5-fold CV
n_folds = 5
skf = MultilabelStratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
splits = list(skf.split(train_df, y_full))

# Get optimizer with param groups
def get_optimizer(model, lr):
    decay = []
    no_decay = []
    for name, param in model.named_parameters():
        if not param.requires_grad:
            continue
        if any(x in name for x in ['bn', 'norm', 'bias']):
            no_decay.append(param)
        else:
            decay.append(param)
    return optim.AdamW([
        {'params': decay, 'weight_decay': 5e-4},
        {'params': no_decay, 'weight_decay': 0.0}
    ], lr=lr)

# Training function
def train_fold(fold, train_idx, val_idx, build_fn, ckpt_pattern, epochs=15, batch_size=4, lr=2e-4):
    train_fold_df = train_df.iloc[train_idx].reset_index(drop=True)
    val_fold_df = train_df.iloc[val_idx].reset_index(drop=True)
    y_train_fold = train_fold_df[label_cols].values
    y_val_fold = val_fold_df[label_cols].values
    
    # Per-fold pos_weight (mild, clipped)
    pos = (y_train_fold == 1).sum(axis=0)
    neg = (y_train_fold == 0).sum(axis=0)
    pos_weight = torch.tensor(np.minimum(neg / (pos + 1e-6), 3.0), dtype=torch.float32).to(device)
    print(f'Fold {fold} pos_weight: {pos_weight.cpu().numpy()}')
    
    train_ds = MIPDataset(train_fold_df, 'data/mips/train', y_train_fold, train_transform)
    val_ds = MIPDataset(val_fold_df, 'data/mips/train', y_val_fold, val_transform)
    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    
    # Clear cache before model init
    gc.collect()
    torch.cuda.empty_cache()
    print(f'Fold {fold} GPU memory before model: {torch.cuda.memory_allocated()/1e9:.1f}GB')
    
    model = build_fn()
    optimizer = get_optimizer(model, lr)
    scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=epochs)
    criterion = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
    scaler = GradScaler()
    
    best_val_loss = float('inf')
    patience = 5
    counter = 0
    for epoch in range(epochs):
        model.train()
        train_loss = 0.0
        start_time = time.time()
        for batch_idx, (images, labels) in enumerate(tqdm(train_loader, desc=f'Fold {fold} Epoch {epoch+1}')):
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            with autocast():
                logits = model(images)
                loss = criterion(logits, labels)
            scaler.scale(loss).backward()
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            scaler.step(optimizer)
            scaler.update()
            train_loss += loss.item() * images.size(0)
            if batch_idx % 10 == 0:
                print(f'Batch {batch_idx}, elapsed: {time.time() - start_time:.1f}s')
        train_loss /= len(train_ds)
        scheduler.step()
        print(f'Fold {fold} Epoch {epoch+1} train time: {time.time() - start_time:.1f}s')
        
        # Val
        model.eval()
        val_loss = 0.0
        val_start = time.time()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                with autocast():
                    logits = model(images)
                    loss = criterion(logits, labels)
                val_loss += loss.item() * images.size(0)
        val_loss /= len(val_ds)
        print(f'Fold {fold} Epoch {epoch+1}: Train {train_loss:.4f}, Val {val_loss:.4f}, val time: {time.time() - val_start:.1f}s')
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            counter = 0
            torch.save(model.state_dict(), ckpt_pattern.format(fold))
            print(f'New best at epoch {epoch+1}')
        else:
            counter += 1
            if counter >= patience:
                print(f'Early stop at epoch {epoch+1}')
                break
    print(f'Fold {fold} best val loss: {best_val_loss:.4f}')
    # Clear after fold
    del model
    gc.collect()
    torch.cuda.empty_cache()

# Train ConvNeXt v1
print('Training ConvNeXt v1 on full train (pretrained)...')
for fold, (train_idx, val_idx) in enumerate(splits):
    train_fold(fold+1, train_idx, val_idx, build_convnext_v1, 'fold_{}_convnext_pretrained.pth')

# Train ConvNeXt v2
print('Training ConvNeXt v2 on full train (pretrained)...')
for fold, (train_idx, val_idx) in enumerate(splits):
    train_fold(fold+1, train_idx, val_idx, build_convnext_v2, 'fold_{}_convnext_v2_pretrained.pth')

# Train RegNet
print('Training RegNet on full train (pretrained)...')
for fold, (train_idx, val_idx) in enumerate(splits):
    train_fold(fold+1, train_idx, val_idx, build_regnet, 'fold_{}_regnet_pretrained.pth')

print('Retraining complete. New pretrained checkpoints saved as fold_*_pretrained.pth.')
# Next: Add cell to generate TTA OOF logits on full 202 using new models (HFlip only), save oof_logits_*_pretrained.npy
# Compute holdout preds/WLL with new OOF; tune blend weights/temps/gamma/lam/shrink on holdout only (low-DOF, >=0.002 gain)
# Update 03_inference_tta cell 0: change paths to _pretrained.pth, load new postproc.npz, use OR-gamma for overall, HFlip TTA
# Re-execute inference cell 0, submit targeting holdout <=0.42 LB <=0.45 medal. If >0.43 holdout, add coronal model.

Using device: cuda
GPU memory after clear: 0.1GB allocated, 0.1GB reserved
Full train size: 202
Training ConvNeXt v1 on full train (pretrained)...
Fold 1 pos_weight: [3. 3. 3. 3. 3. 3. 3.]
Fold 1 GPU memory before model: 0.1GB


Fold 1 Epoch 1:   0%|          | 0/41 [00:00<?, ?it/s]

Fold 1 Epoch 1:   2%|▏         | 1/41 [00:00<00:12,  3.13it/s]

Fold 1 Epoch 1:  10%|▉         | 4/41 [00:00<00:03, 10.54it/s]

Batch 0, elapsed: 0.3s


...

Fold 5 Epoch 11:  88%|████████▊ | 36/41 [00:01<00:00, 22.17it/s]

Fold 5 Epoch 11:  95%|█████████▌| 39/41 [00:01<00:00, 22.05it/s]

Fold 5 Epoch 11: 100%|██████████| 41/41 [00:01<00:00, 22.23it/s]

Batch 40, elapsed: 1.8s
Fold 5 Epoch 11 train time: 1.8s


Fold 5 Epoch 11: Train 0.7098, Val 0.6792, val time: 0.2s
Early stop at epoch 11
Fold 5 best val loss: 0.6772
Retraining complete. New pretrained checkpoints saved as fold_*_pretrained.pth.


In [3]:
# Generate TTA OOF Logits for Pretrained Models (HFlip only, 320x320 to match training)
# Save as oof_logits_convnext_pretrained.npy, etc. for downstream blending/postproc
import torch
import timm
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
from tqdm import tqdm
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
N_FOLDS = 5
IMG_SIZE = 320  # Match training size
label_cols = ['C1','C2','C3','C4','C5','C6','C7']
train_df = pd.read_csv('data/train_mips.csv')
mip_dir_train = 'data/mips/train'
class TrainMIPDataset(Dataset):
    def __init__(self, df, mip_dir, transform):
        self.df = df.reset_index(drop=True)
        self.mip_dir = mip_dir
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        uid = self.df.iloc[idx]['StudyInstanceUID']
        mip = np.load(os.path.join(self.mip_dir, f'{uid}.npy')).astype(np.float32)
        img = np.transpose(mip, (1,2,0))
        img = self.transform(image=img)['image']
        return uid, img
oof_transform = A.Compose([A.Resize(IMG_SIZE, IMG_SIZE), A.Normalize(mean=0.5, std=0.5), ToTensorV2()])
oof_loader = DataLoader(TrainMIPDataset(train_df, mip_dir_train, oof_transform),
                        batch_size=16, shuffle=False, num_workers=0, pin_memory=True)
def build_convnext_v1():
    return timm.create_model('convnext_tiny', pretrained=False, num_classes=7, in_chans=3,
                             drop_rate=0.25, drop_path_rate=0.1).to(device).eval()
def build_convnext_v2():
    return timm.create_model('convnext_tiny', pretrained=False, num_classes=7, in_chans=3,
                             drop_rate=0.25, drop_path_rate=0.1).to(device).eval()
def build_regnet():
    return timm.create_model('regnety_004', pretrained=False, num_classes=7, in_chans=3,
                             drop_rate=0.25, drop_path_rate=0.1).to(device).eval()
def predict_oof_backbone(ckpt_pattern, build_fn):
    all_logits = []
    for f in range(1, N_FOLDS+1):
        ckpt = ckpt_pattern.format(f)
        if not os.path.exists(ckpt):
            print(f'Skip missing {ckpt}'); continue
        model = build_fn()
        sd = torch.load(ckpt, map_location='cpu', weights_only=True)
        model.load_state_dict(sd, strict=True)
        fold_logits = []
        with torch.no_grad():
            for uids, images in tqdm(oof_loader, desc=f'{ckpt_pattern} F{f}'):
                images = images.to(device)
                logits = model(images)
                logits_f = model(torch.flip(images, dims=[3]))  # HFlip TTA
                logits = 0.5 * (logits + logits_f)
                fold_logits.append(logits.cpu().numpy())
        all_logits.append(np.concatenate(fold_logits, axis=0))
    if len(all_logits) == 0:
        raise RuntimeError(f'No checkpoints for {ckpt_pattern}')
    return np.mean(np.stack(all_logits, 0), 0)
print('Generating OOF logits for ConvNeXt v1 pretrained...')
oof_logits_v1 = predict_oof_backbone('fold_{}_convnext_pretrained.pth', build_convnext_v1)
np.save('oof_logits_convnext_pretrained.npy', oof_logits_v1)
print('Generating OOF logits for ConvNeXt v2 pretrained...')
oof_logits_v2 = predict_oof_backbone('fold_{}_convnext_v2_pretrained.pth', build_convnext_v2)
np.save('oof_logits_convnext_v2_pretrained.npy', oof_logits_v2)
print('Generating OOF logits for RegNet pretrained...')
oof_logits_reg = predict_oof_backbone('fold_{}_regnet_pretrained.pth', build_regnet)
np.save('oof_logits_regnet_pretrained.npy', oof_logits_reg)
print('OOF logits saved: oof_logits_*_pretrained.npy. Next: insert cell to compute new 3-way OOF WLL (target <=0.42), holdout preds/WLL; if holdout <=0.42, tune postproc on holdout (low DOF); update 03_inference_tta with new paths/320 size/HFlip TTA, execute for submission targeting LB <=0.45 medal. If holdout >0.43, request expert on coronal/Swin add or further reg.')

Generating OOF logits for ConvNeXt v1 pretrained...


fold_{}_convnext_pretrained.pth F1:   0%|          | 0/13 [00:00<?, ?it/s]

fold_{}_convnext_pretrained.pth F1:   8%|▊         | 1/13 [00:00<00:02,  5.54it/s]

fold_{}_convnext_pretrained.pth F1:  15%|█▌        | 2/13 [00:00<00:01,  6.05it/s]

fold_{}_convnext_pretrained.pth F1:  23%|██▎       | 3/13 [00:00<00:01,  6.62it/s]

...

fold_{}_regnet_pretrained.pth F5:  62%|██████▏   | 8/13 [00:00<00:00, 15.35it/s]

fold_{}_regnet_pretrained.pth F5:  77%|███████▋  | 10/13 [00:00<00:00, 15.34it/s]

fold_{}_regnet_pretrained.pth F5:  92%|█████████▏| 12/13 [00:00<00:00, 15.37it/s]

fold_{}_regnet_pretrained.pth F5: 100%|██████████| 13/13 [00:00<00:00, 15.85it/s]

OOF logits saved: oof_logits_*_pretrained.npy. Next: insert cell to compute new 3-way OOF WLL (target <=0.42), holdout preds/WLL; if holdout <=0.42, tune postproc on holdout (low DOF); update 03_inference_tta with new paths/320 size/HFlip TTA, execute for submission targeting LB <=0.45 medal. If holdout >0.43, request expert on coronal/Swin add or further reg.


In [4]:
# Compute New 3-Way OOF WLL and Holdout WLL for Pretrained Models
# Use existing blend weights/temps/LR/alpha; target OOF <=0.42 (from 0.4211), holdout <=0.42 (from 0.4343)
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold
from scipy.special import expit as sigmoid
from scipy.optimize import minimize_scalar
import joblib
from torch.utils.data import Dataset, DataLoader
import albumentations as A
from albumentations.pytorch import ToTensorV2
import torch
from tqdm import tqdm
import os
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
label_cols = ['C1','C2','C3','C4','C5','C6','C7']
N_FOLDS = 5
IMG_SIZE = 320
# Load OOF labels y8
train_df = pd.read_csv('data/train_mips.csv')
train_labels = pd.read_csv('train.csv')
y_df = train_df[['StudyInstanceUID']].merge(train_labels, on='StudyInstanceUID', how='left')
y7 = y_df[label_cols].values.astype(float)
y_overall = (y7.max(axis=1) > 0).astype(int)
y8 = np.hstack([y7, y_overall[:, None]])
# Load new pretrained OOF logits
v1 = np.load('oof_logits_convnext_pretrained.npy')
v2 = np.load('oof_logits_convnext_v2_pretrained.npy')
reg = np.load('oof_logits_regnet_pretrained.npy')
# Load existing 3-way artifacts
W = np.load('weights_threeway_tta.npy')  # [7,3]
T7 = np.load('temperatures_weighted_tta.npy')  # [7]
rule = open('overall_rule_tta.txt').read().strip()
alpha = float(np.load('overall_lr_alpha_tta.npy'))
# Blend to X [N,7]
stack = np.stack([v1, v2, reg], axis=2)
X = np.sum(stack * W[None, :, :], axis=2)
p7_uncal = sigmoid(X)
p7_cal = sigmoid(X / T7)
union = 1 - np.prod(1 - p7_uncal, axis=1)
max_prob = p7_uncal.max(axis=1)
base_overall = union if rule == 'union' else max_prob
# CV p_lr_oof
skf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=42)
p_lr_oof = np.zeros(len(X))
for tr_idx, va_idx in skf.split(X, y7):
    lr = LogisticRegression(penalty='l2', C=1.0, class_weight='balanced',
                            solver='lbfgs', max_iter=1000, random_state=42)
    lr.fit(X[tr_idx], y_overall[tr_idx])
    p_lr_oof[va_idx] = lr.predict_proba(X[va_idx])[:, 1]
p_overall = alpha * p_lr_oof + (1 - alpha) * base_overall
p8 = np.hstack([p7_cal, p_overall[:, None]])
def wll(y_true8, p8):
    p8 = np.clip(p8, 1e-6, 1 - 1e-6)
    losses = [log_loss(y_true8[:, i], p8[:, i]) for i in range(8)]
    return np.average(losses, weights=np.array([1] * 7 + [2]))
oof_wll = wll(y8, p8)
print(f'New pretrained 3-way OOF WLL: {oof_wll:.4f} (prev 0.4211)')
# Now predict on holdout (41 samples)
holdout_df = pd.read_csv('data/holdout.csv')
holdout_labels = pd.read_csv('train.csv')
y_holdout_df = holdout_df[['StudyInstanceUID']].merge(holdout_labels, on='StudyInstanceUID', how='left')
y_holdout7 = y_holdout_df[label_cols].values.astype(float)
y_holdout_overall = (y_holdout7.max(axis=1) > 0).astype(int)
y_holdout8 = np.hstack([y_holdout7, y_holdout_overall[:, None]])
class HoldoutMIPDataset(Dataset):
    def __init__(self, df, mip_dir, transform):
        self.df = df.reset_index(drop=True)
        self.mip_dir = mip_dir
        self.transform = transform
    def __len__(self): return len(self.df)
    def __getitem__(self, idx):
        uid = self.df.iloc[idx]['StudyInstanceUID']
        mip = np.load(os.path.join(self.mip_dir, f'{uid}.npy')).astype(np.float32)
        img = np.transpose(mip, (1, 2, 0))
        img = self.transform(image=img)['image']
        return uid, img
holdout_transform = A.Compose([A.Resize(IMG_SIZE, IMG_SIZE), A.Normalize(mean=0.5, std=0.5), ToTensorV2()])
holdout_loader = DataLoader(HoldoutMIPDataset(holdout_df, 'data/mips/train', holdout_transform),
                            batch_size=4, shuffle=False, num_workers=0, pin_memory=True)
def predict_holdout_backbone(ckpt_pattern, build_fn):
    all_logits = []
    for f in range(1, N_FOLDS + 1):
        ckpt = ckpt_pattern.format(f)
        if not os.path.exists(ckpt):
            print(f'Skip missing {ckpt}'); continue
        model = build_fn()
        sd = torch.load(ckpt, map_location='cpu', weights_only=True)
        model.load_state_dict(sd, strict=True)
        model.eval()
        fold_logits = []
        with torch.no_grad():
            for uids, images in tqdm(holdout_loader, desc=f'{ckpt_pattern} F{f}'):
                images = images.to(device)
                logits = model(images)
                logits_f = model(torch.flip(images, dims=[3]))  # HFlip TTA
                logits = 0.5 * (logits + logits_f)
                fold_logits.append(logits.cpu().numpy())
        all_logits.append(np.concatenate(fold_logits, axis=0))
    return np.mean(np.stack(all_logits, 0), 0)
print('Generating holdout logits for ConvNeXt v1 pretrained...')
holdout_logits_v1 = predict_holdout_backbone('fold_{}_convnext_pretrained.pth', build_convnext_v1)
print('Generating holdout logits for ConvNeXt v2 pretrained...')
holdout_logits_v2 = predict_holdout_backbone('fold_{}_convnext_v2_pretrained.pth', build_convnext_v2)
print('Generating holdout logits for RegNet pretrained...')
holdout_logits_reg = predict_holdout_backbone('fold_{}_regnet_pretrained.pth', build_regnet)
# Blend holdout X
holdout_stack = np.stack([holdout_logits_v1, holdout_logits_v2, holdout_logits_reg], axis=2)
holdout_X = np.sum(holdout_stack * W[None, :, :], axis=2)
holdout_p7_uncal = sigmoid(holdout_X)
holdout_p7_cal = sigmoid(holdout_X / T7)
holdout_union = 1 - np.prod(1 - holdout_p7_uncal, axis=1)
holdout_max_prob = holdout_p7_uncal.max(axis=1)
holdout_base_overall = holdout_union if rule == 'union' else holdout_max_prob
# Full LR for holdout (use full train X/y_overall)
lr_full = LogisticRegression(penalty='l2', C=1.0, class_weight='balanced',
                             solver='lbfgs', max_iter=1000, random_state=42)
lr_full.fit(X, y_overall)
holdout_p_lr = lr_full.predict_proba(holdout_X)[:, 1]
holdout_p_overall = alpha * holdout_p_lr + (1 - alpha) * holdout_base_overall
holdout_p8 = np.hstack([holdout_p7_cal, holdout_p_overall[:, None]])
holdout_wll = wll(y_holdout8, holdout_p8)
print(f'New pretrained 3-way holdout WLL: {holdout_wll:.4f} (prev 0.4343)')
# If holdout_wll <= 0.42, next: tune postproc on holdout (lam/gamma/shrink, low DOF, stability gate); save postproc_holdout_pretrained.npz
# Update 03_inference_tta cell 0: ckpt *_pretrained.pth, IMG_SIZE=320, batch_size=16, load new postproc.npz, HFlip TTA; execute for submission
# If holdout_wll > 0.43, request expert on adding Swin-T or coronal MIPs for diversity.

New pretrained 3-way OOF WLL: 0.4484 (prev 0.4211)
Generating holdout logits for ConvNeXt v1 pretrained...


fold_{}_convnext_pretrained.pth F1:   0%|          | 0/11 [00:00<?, ?it/s]

fold_{}_convnext_pretrained.pth F1:  36%|███▋      | 4/11 [00:00<00:00, 30.92it/s]

fold_{}_convnext_pretrained.pth F1:  73%|███████▎  | 8/11 [00:00<00:00, 31.17it/s]

fold_{}_convnext_pretrained.pth F1: 100%|██████████| 11/11 [00:00<00:00, 32.63it/s]

...

fold_{}_regnet_pretrained.pth F5:   0%|          | 0/11 [00:00<?, ?it/s]

fold_{}_regnet_pretrained.pth F5:  45%|████▌     | 5/11 [00:00<00:00, 41.38it/s]

fold_{}_regnet_pretrained.pth F5:  91%|█████████ | 10/11 [00:00<00:00, 41.21it/s]

fold_{}_regnet_pretrained.pth F5: 100%|██████████| 11/11 [00:00<00:00, 41.86it/s]

New pretrained 3-way holdout WLL: 0.4520 (prev 0.4343)
